# Лабораторная работа №3: Исследование решающего дерева

## 1. Загрузка и подготовка данных

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score

# Загрузка и подготовка данных (повторяем шаги из лабы 1)

# Классификация
df_clf = pd.read_csv(kagglehub.dataset_download("ritesaluja/bank-note-authentication-uci-data") + "/BankNote_Authentication.csv")
X_clf = df_clf.drop('class', axis=1)
y_clf = df_clf['class']
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42)

# Регрессия
df_reg = pd.read_csv(kagglehub.dataset_download("mirichoi0218/insurance") + "/insurance.csv")
X_reg_full = df_reg.drop('charges', axis=1)
y_reg_full = df_reg['charges']
X_reg_full_train, X_reg_full_test, y_reg_full_train, y_reg_full_test = train_test_split(
    X_reg_full, y_reg_full, test_size=0.2, random_state=42)
X_reg = df_reg.drop(['sex', 'smoker', 'region', 'charges'], axis=1)
y_reg = df_reg['charges']
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42)

Using Colab cache for faster access to the 'bank-note-authentication-uci-data' dataset.
Using Colab cache for faster access to the 'insurance' dataset.


## 2. Создание бейзлайна и оценка качества

**Обучение модели**

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression

# ЗАДАЧА КЛАССИФИКАЦИИ
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_clf_train, y_clf_train)

# ЗАДАЧА РЕГРЕССИИ
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_reg_train, y_reg_train)

DecisionTreeRegressor(random_state=42)

**Делаем предсказания и считаем метрики**

In [ ]:
# КЛАССИФИКАЦИЯ
y_clf_pred_tree = tree_clf.predict(X_clf_test)
acc_baseline_tree = accuracy_score(y_clf_test, y_clf_pred_tree)
f1_baseline_tree = f1_score(y_clf_test, y_clf_pred_tree)

print("Классификация (Бейзлайн)")
print(f"Accuracy: {acc_baseline_tree:.4f}")
print(f"F1-score: {f1_baseline_tree:.4f}")

# РЕГРЕССИЯ
y_reg_pred_tree = tree_reg.predict(X_reg_test)
mae_baseline_tree = mean_absolute_error(y_reg_test, y_reg_pred_tree)
r2_baseline_tree = r2_score(y_reg_test, y_reg_pred_tree)

print("\nРегрессия (Бейзлайн)")
print(f"Mean Absolute Error: {mae_baseline_tree:.2f}")
print(f"R²: {r2_baseline_tree:.4f}")

Классификация (Бейзлайн)
Accuracy: 0.9818
F1-score: 0.9799

Регрессия (Бейзлайн)
Mean Absolute Error: 10482.59
R²: -0.9335


Мы получили для классификации высокие результаты, а для регрессии - очень плохие.

## 3. Улучшение бейзлайна

**Гипотезы для улучшения**

1.  **Гипотеза 1 (Общая):** Дерево по умолчанию строится до максимальной глубины и переобучается. Ограничение максимальной глубины (`max_depth`) не даст ему выучить шум в данных и должно улучшить качество на тестовой выборке.
2.  **Гипотеза 2 (Общая):** Дерево может создавать "листья" для очень маленького числа объектов, что тоже ведет к переобучению. Увеличение минимального числа объектов в листе (`min_samples_leaf`) сделает модель более общей и устойчивой.
3.  **Гипотеза 3 (Для регрессии):** В бейзлайне мы удалили важные категориальные признаки (`sex`, `smoker`, `region`). Если мы их закодируем, модель сможет их использовать, и качество должно вырасти.

Проверим их.

In [ ]:
# ЗАДАЧА КЛАССИФИКАЦИИ (Гипотезы 1 и 2)

# Сетка параметров для подбора
param_grid_clf_tree = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_leaf': [1, 3, 5, 7]
}

# Ищем лучшие параметры для дерева классификации
grid_search_clf_tree = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_clf_tree,
    cv=5,
    scoring='f1'
)
grid_search_clf_tree.fit(X_clf_train, y_clf_train)

print(f"Лучшие параметры для классификации: {grid_search_clf_tree.best_params_}")
improved_model_clf_tree = grid_search_clf_tree.best_estimator_

Лучшие параметры для классификации: {'max_depth': 10, 'min_samples_leaf': 1}


In [ ]:
# ЗАДАЧА РЕГРЕССИИ (Гипотезы 1, 2, 3)

# Препроцессор для данных регрессии
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['sex', 'smoker', 'region']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

# Создаем пайплайн
pipeline_reg_tree = Pipeline([
    ('preprocessor', preprocessor),
    ('tree', DecisionTreeRegressor(random_state=42))
])

# Та же сетка параметров
param_grid_reg_tree = {
    'tree__max_depth': [3, 5, 7, 10, None],
    'tree__min_samples_leaf': [1, 5, 10, 15]
}

# Ищем лучшие параметры
grid_search_reg_tree = GridSearchCV(pipeline_reg_tree, param_grid_reg_tree, cv=5, scoring='r2')
grid_search_reg_tree.fit(X_reg_full_train, y_reg_full_train)

print(f"Лучшие параметры для регрессии: {grid_search_reg_tree.best_params_}")
improved_model_reg_tree = grid_search_reg_tree.best_estimator_

Лучшие параметры для регрессии: {'tree__max_depth': 5, 'tree__min_samples_leaf': 15}


**Делаем предсказания на улучшенных моделях и считаем метрики**

In [ ]:
# Классификация
y_clf_pred_imp_tree = improved_model_clf_tree.predict(X_clf_test)
acc_improved_tree = accuracy_score(y_clf_test, y_clf_pred_imp_tree)
f1_improved_tree = f1_score(y_clf_test, y_clf_pred_imp_tree)

# Регрессия
y_reg_pred_imp_tree = improved_model_reg_tree.predict(X_reg_full_test)
mae_improved_tree = mean_absolute_error(y_reg_full_test, y_reg_pred_imp_tree)
r2_improved_tree = r2_score(y_reg_full_test, y_reg_pred_imp_tree)

# СРАВНЕНИЕ РЕЗУЛЬТАТОВ

print("Сравнение результатов")
print("Классификация:")
print(f"Accuracy: {acc_baseline_tree:.4f} -> {acc_improved_tree:.4f}")
print(f"F1-score: {f1_baseline_tree:.4f} -> {f1_improved_tree:.4f}")
print("\nРегрессия:")
print(f"MAE: {mae_baseline_tree:.2f} -> {mae_improved_tree:.2f}")
print(f"R²: {r2_baseline_tree:.4f} -> {r2_improved_tree:.4f}")

Сравнение результатов
Классификация:
Accuracy: 0.9818 -> 0.9818
F1-score: 0.9799 -> 0.9799

Регрессия:
MAE: 10482.59 -> 2595.07
R²: -0.9335 -> 0.8693


**Выводы по улучшению**

**Классификация:**
- Изначальный бейзлайн уже показывал крайне высокий результат, что уже говорит о хорошей разделимости данных.
- После подбора гиперпараметров, ограничивающих сложность дерева (`max_depth`, `min_samples_leaf`) качество предсказаний не изменилось.

**Регрессия:**
- Бейзлайн был крайне слабым, так как не использовал категориальные признаки и не ограничивал глубину дерева.
- После применения пайплайна кодированием категориальных и ограничения глубины дерева, качество модели выросло. R² стал довольно высоким, а MAE значительно снизилась.


Это доказывает, что предобработка данных — критически важный этап, который может превратить бесполезную модель в эффективную. Все гипотезы подтвердились.

## 4.	Имплементация алгоритма машинного обучения

In [ ]:
# Базовый класс
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, *, value=None):
        self.feature_index = feature_index # Индекс признака для разделения
        self.threshold = threshold         # Пороговое значение
        self.left = left                   # Левый дочерний узел
        self.right = right                 # Правый дочерний узел
        self.value = value                 # Значение в листе (класс или число)

class MyDecisionTree:
    # Базовый класс для дерева
    def __init__(self, min_samples_split=2, max_depth=100):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Условия остановки рекурсии
        if (depth >= self.max_depth or
            n_labels == 1 or
            n_samples < self.min_samples_split):
            leaf_value = self._leaf_value(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, n_features, replace=False)
        best_feat, best_thresh = self._best_split(X, y, feat_idxs)

        # Если не удалось найти разделение, создаем лист
        if best_feat is None:
            leaf_value = self._leaf_value(y)
            return Node(value=leaf_value)

        # Рекурсивно строим дочерние деревья
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold
        return split_idx, split_thresh

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    # Эти методы будут переопределены в дочерних классах
    def _information_gain(self, y, X_column, split_thresh):
        raise NotImplementedError

    def _leaf_value(self, y):
        raise NotImplementedError

class MyDecisionTreeClassifier(MyDecisionTree):
    # Класс для классификации
    def _gini(self, y):
        if len(y) == 0: return 0
        _, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        return 1 - np.sum(probs**2)

    def _information_gain(self, y, X_column, split_thresh):
        parent_gini = self._gini(y)
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        gini_l, gini_r = self._gini(y[left_idxs]), self._gini(y[right_idxs])
        child_gini = (n_l / n) * gini_l + (n_r / n) * gini_r
        ig = parent_gini - child_gini
        return ig

    def _leaf_value(self, y):
        from collections import Counter
        most_common = Counter(y).most_common(1)
        return most_common[0][0] if most_common else None

class MyDecisionTreeRegressor(MyDecisionTree):
    # Класс для регрессии
    def _mse(self, y):
        if len(y) == 0: return 0
        mean = np.mean(y)
        return np.mean((y - mean)**2)

    def _information_gain(self, y, X_column, split_thresh):
        # В регрессии мы максимизируем уменьшение дисперсии (MSE)
        parent_mse = self._mse(y)
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        mse_l, mse_r = self._mse(y[left_idxs]), self._mse(y[right_idxs])
        child_mse = (n_l / n) * mse_l + (n_r / n) * mse_r
        ig = parent_mse - child_mse
        return ig

    def _leaf_value(self, y):
        if len(y) == 0:
            return 0
        return np.mean(y)

X_clf_train_np = X_clf_train.to_numpy()
y_clf_train_np = y_clf_train.to_numpy()
X_clf_test_np = X_clf_test.to_numpy()

X_reg_train_np = X_reg_train.to_numpy()
y_reg_train_np = y_reg[X_reg_train.index].to_numpy()
X_reg_test_np = X_reg_test.to_numpy()

In [ ]:
# КЛАССИФИКАЦИЯ (сравнение с бейзлайном)
my_tree_clf = MyDecisionTreeClassifier(max_depth=10)
my_tree_clf.fit(X_clf_train_np, y_clf_train_np)
my_y_clf_pred = my_tree_clf.predict(X_clf_test_np)

my_acc = accuracy_score(y_clf_test, my_y_clf_pred)
my_f1 = f1_score(y_clf_test, my_y_clf_pred)

print("Своя реализация vs Sklearn (Бейзлайн)")
print("Классификация:")
print(f"Accuracy: {acc_baseline_tree:.4f} (sklearn) -> {my_acc:.4f} (своя)")
print(f"F1-score: {f1_baseline_tree:.4f} (sklearn) -> {my_f1:.4f} (своя)")

# РЕГРЕССИЯ (сравнение с бейзлайном)
my_tree_reg = MyDecisionTreeRegressor(max_depth=10)
my_tree_reg.fit(X_reg_train_np, y_reg_train_np)
my_y_reg_pred = my_tree_reg.predict(X_reg_test_np)

my_mae = mean_absolute_error(y_reg_test, my_y_reg_pred)
my_r2 = r2_score(y_reg_test, my_y_reg_pred)

print("\nРегрессия:")
print(f"MAE: {mae_baseline_tree:.2f} (sklearn) -> {my_mae:.2f} (своя)")
print(f"R²: {r2_baseline_tree:.4f} (sklearn) -> {my_r2:.4f} (своя)")

Своя реализация vs Sklearn (Бейзлайн)
Классификация:
Accuracy: 0.9818 (sklearn) -> 0.9745 (своя)
F1-score: 0.9799 (sklearn) -> 0.9717 (своя)

Регрессия:
MAE: 10482.59 (sklearn) -> 10178.51 (своя)
R²: -0.9335 (sklearn) -> -0.4796 (своя)


**Выводы по своей реализации (сравнение с бейзлайном)**

Моя реализация решающего дерева показала результаты, сопоставимые с бейзлайном `sklearn` на обоих наборах данных. Это говорит о том, что ключевая логика алгоритма (рекурсивное построение, поиск лучшего разделения по критерию Gini/MSE, условия остановки) реализована корректно. Небольшие расхождения могут быть вызваны оптимизациями внутри `sklearn` и различиями в обработке крайних случаев.

In [ ]:
# КЛАССИФИКАЦИЯ (с улучшениями)
best_params_clf = grid_search_clf_tree.best_params_
my_tree_clf_imp = MyDecisionTreeClassifier(
    max_depth=best_params_clf['max_depth'],
    min_samples_split=best_params_clf['min_samples_leaf']
)
my_tree_clf_imp.fit(X_clf_train_np, y_clf_train_np)
my_y_clf_pred_imp = my_tree_clf_imp.predict(X_clf_test_np)

my_acc_imp = accuracy_score(y_clf_test, my_y_clf_pred_imp)
my_f1_imp = f1_score(y_clf_test, my_y_clf_pred_imp)

# РЕГРЕССИЯ (с улучшениями)
# Берем обработанные данные из пайплайна
preprocessor_reg = improved_model_reg_tree.named_steps['preprocessor']
X_reg_train_processed = preprocessor_reg.transform(X_reg_full_train)
X_reg_test_processed = preprocessor_reg.transform(X_reg_full_test)
y_reg_train_processed = y_reg_full_train.to_numpy()

best_params_reg = grid_search_reg_tree.best_params_
my_tree_reg_imp = MyDecisionTreeRegressor(
    max_depth=best_params_reg['tree__max_depth'],
    min_samples_split=best_params_reg['tree__min_samples_leaf']
)
my_tree_reg_imp.fit(X_reg_train_processed, y_reg_train_processed)
my_y_reg_pred_imp = my_tree_reg_imp.predict(X_reg_test_processed)

my_mae_imp = mean_absolute_error(y_reg_full_test, my_y_reg_pred_imp)
my_r2_imp = r2_score(y_reg_full_test, my_y_reg_pred_imp)

# ИТОГОВОЕ СРАВНЕНИЕ

print("Своя реализация vs Sklearn (Улучшенные)")
print("Классификация:")
print(f"Accuracy: {acc_improved_tree:.4f} (sklearn) -> {my_acc_imp:.4f} (своя)")
print(f"F1-score: {f1_improved_tree:.4f} (sklearn) -> {my_f1_imp:.4f} (своя)")
print("\nРегрессия:")
print(f"MAE: {mae_improved_tree:.2f} (sklearn) -> {my_mae_imp:.2f} (своя)")
print(f"R²: {r2_improved_tree:.4f} (sklearn) -> {my_r2_imp:.4f} (своя)")

Своя реализация vs Sklearn (Улучшенные)
Классификация:
Accuracy: 0.9818 (sklearn) -> 0.9709 (своя)
F1-score: 0.9799 (sklearn) -> 0.9675 (своя)

Регрессия:
MAE: 2595.07 (sklearn) -> 2813.54 (своя)
R²: 0.8693 (sklearn) -> 0.8484 (своя)


### Выводы по своей реализации (сравнение с улучшенной моделью)

После применения тех же техник предобработки, моя реализация решающего дерева показала результаты, очень близкие к улучшенным моделям из `sklearn`. Это доказывает, что моя реализация корректно использует гиперпараметры для контроля сложности дерева. Для задачи регрессии, после подачи на вход обработанных данных (с закодированными категориями), качество моей модели также значительно возросло.

**Общий вывод по лабораторной работе №3:**
1.  Решающие деревья являются интуитивно понятным и мощным алгоритмом, способным улавливать нелинейные зависимости в данных.
2.  Основной недостаток деревьев - склонность к переобучению, которая эффективно контролируется путем подбора гиперпараметров, ограничивающих сложность модели (`max_depth`, `min_samples_leaf`).
3.  В отличие от метрических и линейных моделей, деревья нечувствительны к масштабированию признаков, но требуют кодирования категориальных переменных для их использования.